In [41]:
import json, random

from ollama import chat
from ollama import ChatResponse

with open("../dataset/ideals.json", "r") as f:
    data = json.load(f)

template = '''Name: <name>\nGender: <gender>\nResidency: <residency>\nParty: <party>\nDescription: <who is this person, what are their ideals, and why would they vote for their party.>'''

#targets = list(data.keys())[:5]
targets = ["Australian Labor Party", "Liberal Party of Australia", "National Party", "The Australian Greens", "Katter's Australian Party", "Centre Alliance"]

descriptions = []
for party in targets:
	response: ChatResponse = chat(model='qwen3', messages=[
		{
			'role': 'user',
			'content': f'```\n{template}\n```\n\nPlease use the template above to generate a persona that represent a voter for "{party}".\nUse the following party policies for inspiration:\n{", ".join(data[party])}',
		},
	])
	description = response['message']['content'].replace("**", "").split("Description: ")[-1].split("\n")[0]
	print(f"{party} -> {description}")
	descriptions.append(description)

Australian Labor Party -> Sarah is a 45-year-old nurse working in public healthcare, married with two children attending public school. She is a lifelong advocate for accessible, high-quality healthcare and affordable housing, driven by her experience seeing underfunded clinics and rising living costs strain her community. Sarah aligns with the Australian Labor Party because its policies directly address her priorities: expanding Medicare through bulk billing and urgent care clinics ensures her patients can access care without financial stress, while reducing prescription costs and opening specialized clinics like Endometriosis and Pelvic Pain centers reflect her belief in prioritizing women’s health. She supports Labor’s housing initiatives—such as the Help to Buy program, first home buyer incentives, and social housing upgrades—as critical to stabilizing her own family’s financial future and preventing housing insecurity. Sarah also values the party’s focus on education equity (free 

In [51]:
import numpy as np, base64

with open("../dataset/emb_data_not_delta.json", "r") as f:
    data = json.load(f)

parties = dict([(k, np.frombuffer(base64.b64decode(v), dtype=np.float32)) for (k, v) in data["parties"].items()])
policies = dict([(k, np.frombuffer(base64.b64decode(v), dtype=np.float32)) for (k, v) in data["policies"].items()])

def closest(parties, user_emb):
    distances = {party: 1 - np.dot(user_emb, emb) for party, emb in parties.items()}
    return sorted(distances, key=distances.get)

In [52]:
import random
n = 1

results = {}
total_repeats = 10

for repeat in range(total_repeats):
	for target_party, target_persona in zip(targets, descriptions):

		count = 0
		failure_count = 0
		user_emb = np.zeros_like(list(policies.values())[0])

		while True:
			
			policy = random.choice(list(policies.keys()))

			response: ChatResponse = chat(model='gemma3', messages=[
				{
					'role': 'user',
					'content': f'```{target_persona}\n```\n\nUsing the persona above, how likely is the user to agree with the statement: "{policy}".\nPlease only respond with a percentage from 0% to 100%. Do not include any other text.',
				},
			])

			p = response['message']['content'].split("%")[0]
			if p.isnumeric():
				count += 1
				p = float(p)
				p = ((p/100) - 0.5) * 2
		
				user_emb += policies[policy] * p
				#user_emb = user_emb / np.sqrt(np.sum(user_emb**2))
			
				sorted_parties = closest(parties, user_emb / count)

				#print(sorted_parties.index(target_party))

				if sorted_parties.index(target_party) < 5:
					break

				if count > 50:
					failure_count += 1
					count = 0
					user_emb = np.zeros_like(list(policies.values())[0])
		
		print(f"Took {count} attempts for {target_party}")
		results.setdefault(target_party, {"normal_count": 0, "failure_count": 0})
		results[target_party]["normal_count"] += count / total_repeats
		results[target_party]["failure_count"] += failure_count / total_repeats

print(results)

Took 1 attempts for Australian Labor Party
Took 8 attempts for Liberal Party of Australia
Took 1 attempts for National Party
Took 14 attempts for The Australian Greens
Took 5 attempts for Katter's Australian Party
Took 18 attempts for Centre Alliance
Took 29 attempts for Australian Labor Party
Took 9 attempts for Liberal Party of Australia
Took 1 attempts for National Party
Took 2 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 1 attempts for Centre Alliance
Took 1 attempts for Australian Labor Party
Took 18 attempts for Liberal Party of Australia
Took 1 attempts for National Party
Took 20 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 4 attempts for Centre Alliance
Took 5 attempts for Australian Labor Party
Took 5 attempts for Liberal Party of Australia
Took 9 attempts for National Party
Took 36 attempts for The Australian Greens
Took 16 attempts for Katter's Australian Party
Took 10 attempts for Centre Allia

In [42]:
import numpy as np, base64

with open("../emb_data.json", "r") as f:
    data = json.load(f)

parties = dict([(k, np.frombuffer(base64.b64decode(v), dtype=np.float32)) for (k, v) in data["parties"].items()])
policies = dict([(k, np.frombuffer(base64.b64decode(v), dtype=np.float32)) for (k, v) in data["policies"].items()])

def closest(parties, user_emb):
    distances = {party: 1 - np.dot(user_emb, emb) for party, emb in parties.items()}
    return sorted(distances, key=distances.get)


In [ ]:
import random
n = 1

results = {}
total_repeats = 10

for repeat in range(total_repeats):
	for target_party, target_persona in zip(targets, descriptions):

		count = 0
		failure_count = 0
		user_emb = np.zeros_like(list(policies.values())[0])

		while True:
			
			policy = random.choice(list(policies.keys()))

			response: ChatResponse = chat(model='gemma3', messages=[
				{
					'role': 'user',
					'content': f'```{target_persona}\n```\n\nUsing the persona above, how likely is the user to agree with the statement: "{policy}".\nPlease only respond with a percentage from 0% to 100%. Do not include any other text.',
				},
			])

			p = response['message']['content'].split("%")[0]
			if p.isnumeric():
				count += 1
				p = float(p)
				p = ((p/100) - 0.5) * 2
		
				user_emb += policies[policy] * p
				#user_emb = user_emb / np.sqrt(np.sum(user_emb**2))
			
				sorted_parties = closest(parties, user_emb / count)

				#print(sorted_parties.index(target_party))

				if sorted_parties.index(target_party) < 5:
					break

				if count > 50:
					failure_count += 1
					count = 0
					user_emb = np.zeros_like(list(policies.values())[0])
		
		print(f"Took {count} attempts for {target_party}")
		results.setdefault(target_party, {"normal_count": 0, "failure_count": 0})
		results[target_party]["normal_count"] += count / total_repeats
		results[target_party]["failure_count"] += failure_count / total_repeats

print(results)

Took 13 attempts for Australian Labor Party
Took 9 attempts for Liberal Party of Australia
Took 40 attempts for National Party
Took 20 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 6 attempts for Centre Alliance
Took 1 attempts for Australian Labor Party
Took 2 attempts for Liberal Party of Australia
Took 6 attempts for National Party
Took 14 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 3 attempts for Centre Alliance
Took 1 attempts for Australian Labor Party
Took 1 attempts for Liberal Party of Australia
Took 10 attempts for National Party
Took 27 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 8 attempts for Centre Alliance
Took 1 attempts for Australian Labor Party
Took 30 attempts for Liberal Party of Australia
Took 1 attempts for National Party
Took 14 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 1 attempts for Centre Allia

In [50]:
import random
n = 1

results = {}
total_repeats = 10

for repeat in range(total_repeats):
	for target_party, target_persona in zip(targets, descriptions):

		count = 0
		failure_count = 0
		user_emb = np.zeros_like(list(policies.values())[0])

		while True:
			
			if count>1:
				sorted_policies = closest(policies, user_emb / count)
			else:
				sorted_policies = list(policies.keys())
				random.shuffle(sorted_policies)

			policy = random.choice(sorted_policies[:10])
			#print(policy)

			response: ChatResponse = chat(model='gemma3', messages=[
				{
					'role': 'user',
					'content': f'```{target_persona}\n```\n\nUsing the persona above, how likely is the user to agree with the statement: "{policy}".\nPlease only respond with a percentage from 0% to 100%. Do not include any other text.',
				},
			])

			p = response['message']['content'].split("%")[0]
			if p.isnumeric():
				count += 1
				p = float(p)
				p = ((p/100) - 0.5) * 2
		
				user_emb += policies[policy] * p
				#user_emb = user_emb / np.sqrt(np.sum(user_emb**2))
			
				sorted_parties = closest(parties, user_emb / count)

				#print(sorted_parties.index(target_party))

				if sorted_parties.index(target_party) < 5:
					break

				if count > 50:
					failure_count += 1
					count = 0
					user_emb = np.zeros_like(list(policies.values())[0])
		
		print(f"Took {count} attempts for {target_party}")
		results.setdefault(target_party, {"normal_count": 0, "failure_count": 0})
		results[target_party]["normal_count"] += count / total_repeats
		results[target_party]["failure_count"] += failure_count / total_repeats

print(results)

Took 1 attempts for Australian Labor Party
Took 2 attempts for Liberal Party of Australia
Took 8 attempts for National Party
Took 1 attempts for The Australian Greens
Took 13 attempts for Katter's Australian Party
Took 2 attempts for Centre Alliance
Took 1 attempts for Australian Labor Party
Took 5 attempts for Liberal Party of Australia
Took 1 attempts for National Party
Took 1 attempts for The Australian Greens
Took 1 attempts for Katter's Australian Party
Took 2 attempts for Centre Alliance
Took 1 attempts for Australian Labor Party
Took 14 attempts for Liberal Party of Australia
Took 1 attempts for National Party
Took 3 attempts for The Australian Greens
Took 2 attempts for Katter's Australian Party
Took 2 attempts for Centre Alliance
Took 4 attempts for Australian Labor Party
Took 1 attempts for Liberal Party of Australia
Took 2 attempts for National Party
Took 5 attempts for The Australian Greens
Took 5 attempts for Katter's Australian Party
Took 16 attempts for Centre Alliance
T